<a href="https://colab.research.google.com/github/RoyElkabetz/Text-Summarization-with-Deep-Learning/blob/main/Convert_Pandas_DataFrame_to_torchtext_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
## uncomment if you want to mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [49]:
PATH_DATASET = '/content/gdrive/MyDrive/Datasets/Text/news_summary_more.csv'
my_data = pd.read_csv(PATH_DATASET ,encoding='utf-8')
my_data.drop_duplicates(subset=['text'],inplace=True)#dropping duplicates
my_data.dropna(axis=0,inplace=True)#dropping na
my_data.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [50]:
import torch
from torchtext.data.utils import get_tokenizer
import torchtext.legacy.data as data
from torchtext.legacy.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    """Pytorch legacy Dataset: https://pytorch.org/text/_modules/torchtext/data/dataset.html"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
            examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
                string is a field name, and the Field is the associated field.
            filter_pred (callable or None): use only exanples for which
                filter_pred(example) is true, or use all examples if None.
                Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""
  
    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()
        
        for key, field in fields.items():
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

In [51]:
tokenizer = get_tokenizer("basic_english")
# TEXT = data.Field(sequential=True, tokenize='spacy')
# LABEL = data.Field(sequential=False, dtype=torch.float)
# get tokenizer

# define the text field for the dataset
TEXT = data.Field(sequential=True,
                  lower=True, 
                  tokenize=tokenizer,
                  init_token='<sos>', 
                  eos_token='<eos>',
                  dtype=torch.long)
# define the text field for the dataset
SUMMARY = data.Field(sequential=True,
                    lower=True, 
                    tokenize=tokenizer,
                    init_token='<sos>', 
                    eos_token='<eos>',
                    dtype=torch.long)
#SUMMARY = data.Field(sequential=True, tokenize=tokenizer)
# TEXT.build_vocab(my_data, max_size=25000, vectors="glove.6B.100d") 
#TEXT.build_vocab(my_data, max_size=25000) 
#SUMMARY.build_vocab(my_data)
fields = {'headlines': SUMMARY, 'text': TEXT}

In [35]:
train_ds = DataFrameDataset(my_data, fields)
# valid_ds = DataFrameDataset(valid_df, fields)

In [47]:
len(train_ds)

98360

In [42]:
print("Text: ", " ".join([t for t in train_ds.examples[0].text]))
print("Headline: ", " ".join([t for t in train_ds.examples[0].headlines]))


Text:  saurav kant , an alumnus of upgrad and iiit-b ' s pg program in machine learning and artificial intelligence , was a sr systems engineer at infosys with almost 5 years of work experience . the program and upgrad ' s 360-degree career support helped him transition to a data scientist at tech mahindra with 90% salary hike . upgrad ' s online power learning has powered 3 lakh+ careers .
Headline:  upgrad learner switches to career in ml & al with 90% salary hike


In [52]:
train_dataset, test_dataset = DataFrameDataset(my_data, fields={'text': TEXT, 'headlines': SUMMARY}).split()

In [46]:
len(test_dataset)

29508

In [48]:
def batchify(data, bsz, text_field):
    data = text_field.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [53]:
# get tokenizer
tokenizer = data.utils.get_tokenizer("basic_english")


# download the WikiText2 dataset
train_dataset, test_dataset = DataFrameDataset(my_data, fields={'text': TEXT, 'headlines': SUMMARY}).split()

# create vocabulary
TEXT.build_vocab(train_dataset)
vocab = TEXT.vocab

# split the data into batches
batch_size = 20
train_loader = batchify(train_dataset, batch_size, TEXT)
#val_loader = batchify(val_Wiki2, batch_size, text)
test_loader = batchify(test_dataset, batch_size, TEXT)
display(HTML('<h4>Data loaders shapes:</h4>'))
print(f'The train dataset shape is: {train_loader.shape}')
#print(f'The validation dataset shape is: {val_loader.shape}')
print(f'The test dataset shape is: {test_loader.shape}')

# display data samples
display(HTML('<h4>Display data samples:</h4>'))
n_samples = 2
for i in range(n_samples):
    tokens = train_loader[i]
    print(f'Sample {i}:')
    print(f'Tokens: {list(tokens.cpu().numpy())}')
    print("Text: ", " ".join([vocab.itos[t] for t in tokens]))
    print('\n')

NameError: ignored

In [ ]:
# from torchtext import data

# class DataFrameDataset(data.Dataset):

#     def __init__(self, df, text_field, label_field, is_test=False, **kwargs):
#         fields = [('text', text_field), ('label', label_field)]
#         examples = []
#         for i, row in df.iterrows():
#             label = row.sentiment if not is_test else None
#             text = row.text
#             examples.append(data.Example.fromlist([text, label], fields))

#         super().__init__(examples, fields, **kwargs)

#     @staticmethod
#     def sort_key(ex):
#         return len(ex.text)

#     @classmethod
#     def splits(cls, text_field, label_field, train_df, val_df=None, test_df=None, **kwargs):
#         train_data, val_data, test_data = (None, None, None)

#         if train_df is not None:
#             train_data = cls(train_df.copy(), text_field, label_field, **kwargs)
#         if val_df is not None:
#             val_data = cls(val_df.copy(), text_field, label_field, **kwargs)
#         if test_df is not None:
#             test_data = cls(test_df.copy(), text_field, label_field, True, **kwargs)

#         return tuple(d for d in (train_data, val_data, test_data) if d is not None)
        
# train_ds, val_ds, test_ds = DataFrameDataset.splits(
# text_field=TEXT_FIELD, label_field=LABEL_FIELD, train_df=train_df, val_df=val_df, test_df=test_df)